# Using `dask` with Scanpy

`dask` is a popular out-of-core, distributed array processing library that scanpy is beginning to support.  Here we walk through a quick tutorial of using `dask` in a simple analysis task.

In [1]:
import numpy as np
import dask.distributed as dd
import dask.array as da
from dask import delayed
import scanpy as sc
import anndata as ad
import h5py
from xarray.backends import CachingFileManager
from pathlib import Path
sc.logging.print_header()

scanpy==1.10.0rc2.dev93+g31669098 anndata==0.11.0.dev78+g64ab900 umap==0.5.5 numpy==1.26.3 scipy==1.12.0 pandas==2.2.0 scikit-learn==1.3.2 statsmodels==0.14.1 igraph==0.10.8 pynndescent==0.5.11


In [2]:
if not Path("c.h5ad").exists():
    !wget https://datasets.cellxgene.cziscience.com/82eac9c1-485f-4e21-ab21-8510823d4f6e.h5ad -O "cell_atlas.h5ad"

--2024-03-20 15:17:40--  https://datasets.cellxgene.cziscience.com/82eac9c1-485f-4e21-ab21-8510823d4f6e.h5ad
Resolving datasets.cellxgene.cziscience.com (datasets.cellxgene.cziscience.com)... 18.172.112.87, 18.172.112.108, 18.172.112.61, ...
Connecting to datasets.cellxgene.cziscience.com (datasets.cellxgene.cziscience.com)|18.172.112.87|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13979243997 (13G) [binary/octet-stream]
Saving to: ‘cell_atlas.h5ad’

cell_atlas.h5ad     100%[===================>]  13,02G  24,5MB/s    in 9m 21s  

2024-03-20 15:27:03 (23,7 MB/s) - ‘cell_atlas.h5ad’ saved [13979243997/13979243997]



For more information on using distributed computing via `dask`, please see their [documentation](https://docs.dask.org/en/stable/deploying-python.html).  For example, `dask` provides direct support for [slurm](https://jobqueue.dask.org/en/latest/).  In short, one needs to define both a cluster and a client to have some degree of control over the compute resources dask will use.

In [3]:
cluster = dd.LocalCluster()
client = dd.Client(cluster)
chunksize = 1000

We'll convert the `X` representation to `dask`.  For more info on i/o from disk, please see the `anndata` tutorials, e.g. [here](https://anndata.readthedocs.io/en/latest/tutorials/notebooks/anndata_dask_array.html) or [here](https://anndata.readthedocs.io/en/latest/tutorials/notebooks/%7Bread%2Cwrite%7D_dispatched.html).  For now, simply converting `X` will be enough to demonstrate the functionality of scanpy with dask.

**_Important Note:_** At the moment, scanpy only works with dense, `np.array` chunks in `dask` arrays.  Sparse support is a work in progress, and this tutorial will be updated accordingly when possible. 
**_Important NO


In [4]:
def read_dask(file):
    f = h5py.File(file, 'r')
    manager = CachingFileManager(h5py.File, file, mode='r')
    def callback(func, elem_name: str, elem, iospec):
        if iospec.encoding_type in (
            "dataframe",
            "awkward-array",
        ):
            # Preventing recursing inside of these types
            return ad.experimental.read_elem(elem)
        elif iospec.encoding_type == "array":
            return da.from_array(elem)
        elif iospec.encoding_type in ("csr_matrix", "csc_matrix"):
            shape = elem.attrs["shape"]
            def make_dask_chunk(block_id=None):
                with manager.acquire_context(needs_lock=False) as f:
                    mtx = ad.experimental.sparse_dataset(f[elem_name])
                    (row, _) = block_id
                    chunk = np.asarray(mtx[slice(row * chunksize, min((row * chunksize) + chunksize, shape[0]))].todense())
                return chunk
            chunks_0 = (chunksize,) * (shape[0] // chunksize)
            chunks_0 += (shape[0] % chunksize , )
            chunks_1 = (shape[1],)
            da_mtx = da.map_blocks(
                make_dask_chunk,
                dtype=elem["data"].dtype,
                chunks=(chunks_0, chunks_1),
                meta=np.array([])
            )
            return da_mtx
        return func(elem)
    
    adata = ad.experimental.read_dispatched(f, callback=callback)
    return adata
adata_dask = read_dask('cell_atlas.h5ad')

In [5]:
adata_dask.X

dask.array<make_dask_chunk, shape=(1462702, 27714), dtype=float32, chunksize=(1000, 27714), chunktype=numpy.ndarray>

In [ ]:
sc.pp.filter_cells(adata_dask, min_genes=200)
sc.pp.filter_genes(adata_dask, min_cells=3)

In [ ]:
sc.pp.normalize_total(adata_dask, target_sum=1e4)
sc.pp.highly_variable_genes(adata_dask, min_mean=0.0125, max_mean=3, min_disp=0.5)

In [ ]:
sc.pl.highly_variable_genes(adata_dask)